In [1]:
#Import packages
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

## Load Data Files

In [2]:
adEvents  = pd.read_csv("../Data/Raw Data/adevents.csv", sep=",", index_col='DEIDNUM')
quality = pd.read_csv("../Data/Raw Data/quality.csv", sep=",", index_col='DEIDNUM')
patient  = pd.read_csv("../Data/Raw Data/patient.csv", sep=",", index_col='DEIDNUM')
mhist  = pd.read_csv("../Data/Raw Data/mhist.csv", sep=",", index_col='DEIDNUM')
physical  = pd.read_csv("../Data/Raw Data/physical.csv", sep=",", index_col='DEIDNUM')

hemoOrig = pd.read_csv("../Data/Raw Data/hemo.csv", sep=",", index_col='DEIDNUM')

ex = pd.read_csv("../Data/Raw Data/exercise.csv", sep=",", index_col='DEIDNUM')
labs = pd.read_csv("../Data/Raw Data/labs.csv", sep=",", index_col='DEIDNUM')
meds = pd.read_csv("../Data/Raw Data/meds.csv", sep=",", index_col='DEIDNUM').fillna(0)

## Make Outcome Labels

In [3]:
dthList =  []
rhspList = []
idx = sorted(set(patient.index))
for i in idx:
    t = patient.loc[i]
    #Death outcome
    dth = t["DEATH"] + t['LVADANY'] + t['TRANSP']
    if dth >= 1:
        dthList.append(1)
    else:
        dthList.append(0)
        
    #Rehosp outcome
    rhsp = t['REHOSP']
    if rhsp >= 1:
        rhspList.append(1)
    else:
        rhspList.append(0)
        

#make readmit labels
reAdList = []
idx = sorted(set(patient.index))
for i in idx:
    t = patient.loc[i]
    adt = t['T2REHSP']  #days to first rehosp
    if adt < 30:
        reAdList.append(1)
    else:
        reAdList.append(0)

In [4]:
labels = pd.DataFrame(columns = ['Death', 'Rehosp', 'Readmission'], index=patient.index)
labels['Death'] = dthList
labels['Rehosp'] = rhspList
labels['Readmission'] = reAdList
labels.head(10)

,Death,Rehosp,Readmission
DEIDNUM,,,
72,1,1,0
81,0,1,0
86,0,0,0
267,1,0,0
814,0,0,0
1020,0,0,0
1059,1,1,0
1084,0,0,0
1262,0,1,0


In [5]:
print( "Death/LVAD/Trans Count", len(labels[labels['Death']==1]))
print("Rehosp Count", len(labels[labels['Rehosp']==1]))
print("Readmit Count", len(labels[labels['Readmission']==1]))

labels.to_csv('../Data/Original DataFrames/Labels.csv', index=True)

Death/LVAD/Trans Count 117
Rehosp Count 247
Readmit Count 80


## Make Meds

In [6]:
grandList = []
idx = sorted(set(patient.index))
for i in idx:
    lst = []
    t = meds.loc[i]
    #Ace
    aceB = t["ACEB"] + t['BENB'] + t['LISB'] + t['MONB'] + t['VALSAB'] + t['ANGIOTB'] + t['LOSAB'] + t['CANDB'] + t['CAPB'] + t['ENAB'] + t['FOSB'] + t['QUIB'] + t['RAMB'] + t['TRAB']  
    aceD = t["ACED"] + t['BEND'] + t['LISD'] + t['MOND'] + t['VALSAD'] + t['ANGIOTD'] + t['LOSAD'] + t['CANDD'] + t['CAPD'] + t['ENAD'] + t['FOSD'] + t['QUID'] + t['RAMD'] + t['TRAD']
    lst.append(1 if aceB >= 1 else 0)
    lst.append(1 if aceD >= 1 else 0)
    
    #BET
    betB = t['BETB'] + t['BISB'] + t['CARB'] + t['ATEB'] + t['PROB']
    betD = t['BETD'] + t['BISD'] + t['CARD'] + t['ATED'] + t['PROD']
    lst.append(1 if betB >= 1 else 0)
    lst.append(1 if betD >= 1 else 0)
    
    #NIT
    nitB = t['NITB'] + t['DINB'] + t['NIPB'] + t['BUMB'] + t['ETHB'] + t['NIGB'] + t['TOPB']
    nitD = t['NITD'] + t['DIND'] + t['NIPD'] + t['BUMD'] + t['ETHD'] + t['NIGD'] + t['TOPD']
    lst.append(1 if nitB >= 1 else 0)
    lst.append(1 if nitD >= 1 else 0)
    
    #Diuretics
    diurB = t['FURB'] + t['TORB'] + t['METB'] + t['DIURB'] 
    diurD = t['FURD'] + t['TORD'] + t['METD'] + t['DIURD'] 
    lst.append(1 if diurB >= 1 else 0)
    lst.append(1 if diurD >= 1 else 0)
    
    grandList.append(lst)


cols = ['ACE_B', 'ACE_D', 'BET_B', 'BET_D', 'NIT_B', 'NIT_D', 'DIUR_B', 'DIUR_D']
medData = pd.DataFrame(grandList, columns =cols, index=patient.index)

medData.head(10)

,ACE_B,ACE_D,BET_B,BET_D,NIT_B,NIT_D,DIUR_B,DIUR_D
DEIDNUM,,,,,,,,
72,1,1,1,0,1,1,1,1
81,1,1,1,1,0,1,1,1
86,1,1,0,0,1,1,1,1
267,1,1,1,0,0,1,1,1
814,0,1,1,1,0,0,1,1
1020,1,1,0,1,1,1,1,1
1059,1,1,0,0,1,0,1,1
1084,1,1,0,1,0,1,1,1
1262,1,1,0,1,1,1,1,1


## Make Lab Data

In [7]:
labData = pd.DataFrame(columns =[], index=patient.index)

var = ['ALB', 'ALT', 'AST', 'BUN', 'CRT', 'DIAL', 'HEC', 'HEM', 'PLA', 'POT', 'SOD', 'TALB', 'TOTP', 'WBC']

for v in var:
    idx = sorted(set(labs.index))
    base = []
    final = []
    for i in idx:
        t = labs.loc[i]
       
        base.append(t[v+"B"])
        final.append(t[v+"D"])
        
    labData[v+"_B"] = base
    labData[v+"_D"] = final
    
labData.head(5)

,ALB_B,ALB_D,ALT_B,ALT_D,AST_B,AST_D,BUN_B,BUN_D,CRT_B,CRT_D,...,POT_B,POT_D,SOD_B,SOD_D,TALB_B,TALB_D,TOTP_B,TOTP_D,WBC_B,WBC_D
DEIDNUM,,,,,,,,,,,,,,,,,,,,,
72,3.0,NaN,21.0,NaN,24.0,NaN,39.0,49.0,1.5,1.7,...,3.5,3.7,141.0,135.0,0.70,NaN,6.2,NaN,4.50,4.60
81,3.9,3.6,14.0,14.0,18.0,19.0,49.0,32.0,2.1,1.7,...,4.9,4.3,136.0,134.0,1.20,0.9,6.7,6.4,5.10,7.00
86,3.7,3.5,16.0,17.0,21.0,25.0,16.0,22.0,1.6,1.8,...,4.5,4.8,140.0,135.0,0.39,0.4,7.1,7.2,9.05,11.54
267,2.8,NaN,11.0,NaN,42.0,NaN,74.0,43.0,2.8,1.4,...,4.7,4.5,124.0,135.0,0.60,NaN,5.8,NaN,6.60,10.60
814,4.6,4.3,13.0,15.0,27.0,23.0,16.0,18.0,0.8,0.9,...,3.8,4.3,140.0,136.0,1.20,0.9,7.7,7.5,8.40,9.60


## Make Conditions

In [8]:
grandList = []
idx = sorted(set(patient.index))
for i in idx:
    lst = []
    t = mhist.loc[i]

    lst.append(t['AFIB'])
    lst.append(t['ALCHOE'])
    lst.append(t['ANGP'])
    lst.append(t['ARRH'])
    lst.append(t['CARREST'])
    lst.append(t['CVD'])
    lst.append(t['COPD'])
    lst.append(t['DEPR'])
    lst.append(t['DIAB'])
    lst.append(t['GOUT'])
    lst.append(t['HEPT'])
    lst.append(t['HTN'])
    lst.append(t['MALIG'])
    lst.append(t['RENALI'])
    lst.append(t['SMOKING'])
    lst.append(t['STERD'])
    
    #Stroke / TIA
    if t['STROKE'] == 1 or t['TIA'] == 1:
        lst.append(1)
    else:
        lst.append(0)
    
    #VAHD
    if t['VAHD'] == 1 or t['PVD'] == 1:
        lst.append(1)
    else:
        lst.append(0)

    lst.append(t['VF'])
    
    #VHD
    vhd = t['AOREG'] + t['AOST'] + t['MTST'] + t['PMRG'] + t['PTREG'] + t['VALVUE']
    lst.append(1 if vhd >= 1 else 0)
    
    #VT
    vt = t['SVT'] + t['TDP']
    lst.append(1 if vt >= 1 else 0)
    
    #ischemic
    isch = t['ISCHD'] + t['ISCHEME'] + t['MI'] + t['CYTOE']
    lst.append(1 if isch >= 1 else 0)
    
    #nonischemic
    nisch = t['FAMILE'] + t['HYPERE'] + t['IDIOPE'] + t['PERIPAE']
    lst.append(1 if nisch >= 1 else 0)
    
    #Devices / procedures
    lst.append(t['CABG'])
    lst.append(1 if t['HTRANS'] >= 1 else 0)
    lst.append(t['ICD'])
    lst.append(t['PACE'])
    lst.append(t['PTCI'])
    
    grandList.append(lst)
    


cols = ['AF', 'AlchE','ANGP','ARRH','CARREST','CVD','COPD','DEPR','DIAB','GOUT','HEPT','HTN','MALIG','RENAL','SMOKING',
        'STERD','StrokeTIA','VAHD','VF','VHD','VT', 'ISCH', 'NonISCH', 'CABG', 'HTRANS', 'ICD', 'PACE', 'PTCI']
conditions = pd.DataFrame(grandList, columns =cols, index=patient.index)

conditions.head(10)

,AF,AlchE,ANGP,ARRH,CARREST,CVD,COPD,DEPR,DIAB,GOUT,...,VF,VHD,VT,ISCH,NonISCH,CABG,HTRANS,ICD,PACE,PTCI
DEIDNUM,,,,,,,,,,,,,,,,,,,,,
72,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0,0,1,0,1.0,1,0.0,1.0,0.0
81,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,0,1,0.0,1,0.0,1.0,0.0
86,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0,0,0,1,0.0,1,0.0,0.0,0.0
267,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0,0,1,0,1.0,1,1.0,1.0,0.0
814,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0,1,1,0,0.0,1,1.0,1.0,1.0
1020,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1,0,0,1,0.0,1,0.0,0.0,0.0
1059,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1,1,0,0,0.0,1,0.0,0.0,0.0
1084,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,1,1,0.0,1,0.0,0.0,0.0
1262,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0,0,1,1,1.0,1,1.0,1.0,1.0


## All Data

In [9]:
#  no hemo, just bps
allData = pd.DataFrame(columns =[], index=patient.index)

#demog
allData['Age'] = patient['AGE']
allData['Gender'] = patient['GENDER']
allData['Race'] = patient['RACE']
allData['Wt_B'] = patient['WTADM']
allData['Wt_D'] = patient['WTDIS']
allData['BMI_B'] = patient['bmiadm']
allData['BMI_D'] = patient['bmidis']

allData['InitialHospDays'] = patient['HOSPDAY']
allData['TotalHospDays'] = patient['HSPDAY']

allData['NYHA_B'] = quality['NYHAB']
allData['NYHA_D'] = quality['NYHAD']
allData['MLHFS'] = quality['MLHFSB']


#conditions
allData = pd.concat([allData, conditions], axis=1)

#ex
allData['SixFtWlk_B'] = ex['FTWLKB']
allData['SixFtWlk_D'] = ex['FTWLKD']
allData['VO2_B'] = ex['VO2B']
allData['VO2_D'] = ex['VO2D']

#labs
allData = pd.concat([allData, labData], axis=1)

# meds
allData = pd.concat([allData, medData], axis=1)


# #Hemo ish
allData['EjF_B'] = patient['EF1']
allData['EjF_D'] = patient['EF1']
allData['BPDIAS_B'] = physical['DIASBPB']
allData['BPDIAS_D'] = physical['DIASBPD']
allData['BPSYS_B'] = physical['SYSBPB']
allData['BPSYS_D'] = physical['SYSBPD']
allData['HR_B'] = physical['HRSUPB']
allData['HR_D'] = physical['HRSUPD']


# #Composite
idx = sorted(set(allData.index))
#PV
pv = []
for i in idx:
    t = allData.loc[i]
    
    val = ( ((t['HEM_B'] / t['HEM_D']) * ((100 - t['HEC_D']) / (100  - t['HEC_B']))) - 1 ) * 100
    pv.append(val)
    
allData['PV'] = pv

#MAP
chg = []
base = []
final = []
for i in idx:
    t = allData.loc[i]

    b = t['BPSYS_B'] + ((2 * t['BPDIAS_B']) / 3)
    f = t['BPSYS_D'] + ((2 * t['BPDIAS_D']) / 3)
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
allData['MAP_B'] = base
allData['MAP_D'] = final
allData['MAP_Chg'] = chg

#PP
chg = []
base = []
final = []
for i in idx:
    t = allData.loc[i]

    b = t['BPSYS_B'] - t['BPDIAS_B']
    f = t['BPSYS_D'] - t['BPDIAS_D']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
allData['PP_B'] = base
allData['PP_D'] = final
allData['PP_Chg'] = chg

#PPP
chg = []
base = []
final = []
for i in idx:
    t = allData.loc[i]

    b = t['PP_B'] / t['BPSYS_B']
    f = t['PP_D'] / t['BPSYS_D']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
allData['PPP_B'] = base
allData['PPP_D'] = final
allData['PPP_Chg'] = chg

#PPRatio
chg = []
base = []
final = []
for i in idx:
    t = allData.loc[i]

    b = t['PP_B'] / t['HR_B']
    f = t['PP_D'] / t['HR_D']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
allData['PPRatio_B'] = base
allData['PPRatio_D'] = final
allData['PPRatio_Chg'] = chg


allData.head(5)

,Age,Gender,Race,Wt_B,Wt_D,BMI_B,BMI_D,InitialHospDays,TotalHospDays,NYHA_B,...,MAP_Chg,PP_B,PP_D,PP_Chg,PPP_B,PPP_D,PPP_Chg,PPRatio_B,PPRatio_D,PPRatio_Chg
DEIDNUM,,,,,,,,,,,,,,,,,,,,,
72,88,2,1,57.100000,52.000000,26.424175,24.064047,9.0,16,4.0,...,-13.333333,26.0,26.0,0.0,0.276596,0.302326,0.025730,0.250000,0.282609,0.032609
81,69,1,1,102.363636,100.454545,32.307675,31.705134,5.0,13,4.0,...,-19.666667,32.0,34.0,2.0,0.363636,0.441558,0.077922,0.457143,0.472222,0.015079
86,56,2,2,98.636364,97.000000,40.016375,39.352509,3.0,3,4.0,...,-12.333333,30.0,41.0,11.0,0.300000,0.422680,0.122680,0.348837,0.500000,0.151163
267,82,1,1,82.200000,81.700000,30.938312,30.750122,15.0,14,4.0,...,8.000000,42.0,40.0,-2.0,0.437500,0.400000,-0.037500,0.600000,0.400000,-0.200000
814,58,1,1,82.100000,69.900000,24.515513,20.872525,4.0,4,4.0,...,-4.666667,36.0,48.0,12.0,0.339623,0.444444,0.104822,0.600000,0.750000,0.150000


In [10]:
allData.to_csv('../Data/Original DataFrames/AllData.csv', index=True)

## Hemo

In [11]:
#Calculate hemo changes
hemo = pd.DataFrame(columns = ['RAP_B', 'PAS_B', 'PAD_B', 'PAMN_B', 'PCWP_B',
                               'CO_B', 'CI_B', 'SVR_B', 'MIXED_B', 'BPSYS_B', 'BPDIAS_B', 'HRTRT_B', 'RAT_B',
                               'RAP_D', 'PAS_D', 'PAD_D', 'PAMN_D', 'PCWP_D',  
                               'CO_D', 'CI_D', 'SVR_D', 'MIXED_D', 'BPSYS_D', 'BPDIAS_D', 'HRTRT_D', 'RAT_D',
                               'RAP_Chg', 'PAS_Chg', 'PAD_Chg', 'PAMN_Chg', 'PCWP_Chg', 
                               'CO_Chg', 'CI_Chg', 'SVR_Chg', 'MIXED_Chg', 'BPSYS_Chg', 'BPDIAS_Chg', 
                               'HRTRT_Chg', 'RAT_Chg'], index=patient.index)


var = ['RAP', 'PAS', 'PAD', 'PAMN', 'PCWP', 'CO', 'CI', 'SVR', 'MIXED', 'BPSYS', 
       'BPDIAS', 'HRTRT', 'RAT']

for v in var:
    idx = sorted(set(hemoOrig.index))
    chg = []
    base = []
    final = []
    for i in idx:
        t = hemoOrig.loc[i]
        
        base.append(t[v+"B"])
        final.append(t[v+"L"])
        chg.append(t[v+"L"] - t[v+"B"])
        
    hemo[v+"_B"] = base
    hemo[v+"_D"] = final
    hemo[v+"_Chg"] = chg
    
hemo.head(5)

,RAP_B,PAS_B,PAD_B,PAMN_B,PCWP_B,CO_B,CI_B,SVR_B,MIXED_B,BPSYS_B,...,PAMN_Chg,PCWP_Chg,CO_Chg,CI_Chg,SVR_Chg,MIXED_Chg,BPSYS_Chg,BPDIAS_Chg,HRTRT_Chg,RAT_Chg
DEIDNUM,,,,,,,,,,,,,,,,,,,,,
72,24.0,42.0,24.0,30.0,36.0,2.20,1.60,2354.0,NaN,114.0,...,0.0,-12.0,0.90,0.40,NaN,NaN,-12.0,-8.0,-17.0,0.08
81,10.0,40.0,20.0,27.0,18.0,4.52,2.02,1097.0,60.0,80.0,...,-2.0,-1.0,0.13,0.06,-254.0,-2.0,5.0,-18.0,0.0,0.15
86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
814,3.0,51.0,19.0,30.0,18.0,4.10,2.00,1403.0,97.0,96.0,...,-9.0,-6.0,NaN,NaN,NaN,NaN,-2.0,-10.0,0.0,0.00


In [12]:
#Get only patients with hemo
hemoComposite = hemo.loc[~(np.isnan(hemo)).all(axis=1)]
print(hemoComposite.shape)
hemoComposite.head(5)

(209, 39)


,RAP_B,PAS_B,PAD_B,PAMN_B,PCWP_B,CO_B,CI_B,SVR_B,MIXED_B,BPSYS_B,...,PAMN_Chg,PCWP_Chg,CO_Chg,CI_Chg,SVR_Chg,MIXED_Chg,BPSYS_Chg,BPDIAS_Chg,HRTRT_Chg,RAT_Chg
DEIDNUM,,,,,,,,,,,,,,,,,,,,,
72,24.0,42.0,24.0,30.0,36.0,2.20,1.60,2354.0,NaN,114.0,...,0.0,-12.0,0.90,0.40,NaN,NaN,-12.0,-8.0,-17.0,0.08
81,10.0,40.0,20.0,27.0,18.0,4.52,2.02,1097.0,60.0,80.0,...,-2.0,-1.0,0.13,0.06,-254.0,-2.0,5.0,-18.0,0.0,0.15
814,3.0,51.0,19.0,30.0,18.0,4.10,2.00,1403.0,97.0,96.0,...,-9.0,-6.0,NaN,NaN,NaN,NaN,-2.0,-10.0,0.0,0.00
1084,13.0,40.0,26.0,21.0,20.0,2.91,1.76,1676.0,NaN,88.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1262,10.0,50.0,25.0,33.0,25.0,2.72,1.43,1499.0,NaN,90.0,...,-19.0,-17.0,1.08,0.57,-160.0,NaN,7.0,9.0,34.0,0.10


In [13]:
#Add demog - age, gender, race, ejf
idx = sorted(set(hemoComposite.index))

age = []
gender = []
race = []
ejf = []
for i in idx:
    t = hemoComposite.loc[i]
    age.append(patient.loc[i]['AGE'])
    gender.append(patient.loc[i]['GENDER'])
    race.append(patient.loc[i]['RACE'])
    ejf.append(patient.loc[i]['EF1'])

hemoComposite['Age_B'] = age
hemoComposite['Age_D'] = age
hemoComposite['Gender_B'] = gender
hemoComposite['Gender_D'] = gender
hemoComposite['Race_B'] = race
hemoComposite['Race_D'] = race
hemoComposite['EjF_B'] = ejf
hemoComposite['EjF_D'] = ejf

hemoComposite.head(5)


,RAP_B,PAS_B,PAD_B,PAMN_B,PCWP_B,CO_B,CI_B,SVR_B,MIXED_B,BPSYS_B,...,HRTRT_Chg,RAT_Chg,Age_B,Age_D,Gender_B,Gender_D,Race_B,Race_D,EjF_B,EjF_D
DEIDNUM,,,,,,,,,,,,,,,,,,,,,
72,24.0,42.0,24.0,30.0,36.0,2.20,1.60,2354.0,NaN,114.0,...,-17.0,0.08,88.0,88.0,2.0,2.0,1.0,1.0,25.0,25.0
81,10.0,40.0,20.0,27.0,18.0,4.52,2.02,1097.0,60.0,80.0,...,0.0,0.15,69.0,69.0,1.0,1.0,1.0,1.0,20.0,20.0
814,3.0,51.0,19.0,30.0,18.0,4.10,2.00,1403.0,97.0,96.0,...,0.0,0.00,58.0,58.0,1.0,1.0,1.0,1.0,NaN,NaN
1084,13.0,40.0,26.0,21.0,20.0,2.91,1.76,1676.0,NaN,88.0,...,NaN,NaN,49.0,49.0,2.0,2.0,2.0,2.0,14.0,14.0
1262,10.0,50.0,25.0,33.0,25.0,2.72,1.43,1499.0,NaN,90.0,...,34.0,0.10,66.0,66.0,1.0,1.0,4.0,4.0,12.0,12.0


In [14]:
var = ['MAP', 'MPAP', 'CPI', 'PP', 'PPP', 'PAPP', 'SVR', 'RAT', 'PPRatio']

idx = sorted(set(hemoComposite.index))

#MAP
chg = []
base = []
final = []
for i in idx:
    t = hemoComposite.loc[i]

    b = t['BPSYS_B'] + ((2 * t['BPDIAS_B']) / 3)
    f = t['BPSYS_D'] + ((2 * t['BPDIAS_D']) / 3)
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
hemoComposite['MAP_B'] = base
hemoComposite['MAP_D'] = final
hemoComposite['MAP_Chg'] = chg

#MPAP
chg = []
base = []
final = []
for i in idx:
    t = hemoComposite.loc[i]

    b = t['PAS_B'] + ((2 * t['PAD_B']) / 3)
    f = t['PAS_D'] + ((2 * t['PAD_D']) / 3)
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
hemoComposite['MPAP_B'] = base
hemoComposite['MPAP_D'] = final
hemoComposite['MPAP_Chg'] = chg

#CPI
chg = []
base = []
final = []
for i in idx:
    t = hemoComposite.loc[i]

    b = (t['CI_B'] * t['MAP_B']) / 451
    f = (t['CI_D'] * t['MAP_D']) / 451
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
hemoComposite['CPI_B'] = base
hemoComposite['CPI_D'] = final
hemoComposite['CPI_Chg'] = chg
    
#PP
chg = []
base = []
final = []
for i in idx:
    t = hemoComposite.loc[i]

    b = t['BPSYS_B'] - t['BPDIAS_B']
    f = t['BPSYS_D'] - t['BPDIAS_D']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
hemoComposite['PP_B'] = base
hemoComposite['PP_D'] = final
hemoComposite['PP_Chg'] = chg

#PPP
chg = []
base = []
final = []
for i in idx:
    t = hemoComposite.loc[i]

    b = t['PP_B'] / t['BPSYS_B']
    f = t['PP_D'] / t['BPSYS_D']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
hemoComposite['PPP_B'] = base
hemoComposite['PPP_D'] = final
hemoComposite['PPP_Chg'] = chg
    
#PAPP
chg = []
base = []
final = []
for i in idx:
    t = hemoComposite.loc[i]

    b = (t['PAS_B']  - t['PAD_B']) / t['PAS_B']
    f = (t['PAS_D']  - t['PAD_D']) / t['PAS_D']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
hemoComposite['PAPP_B'] = base
hemoComposite['PAPP_D'] = final
hemoComposite['PAPP_Chg'] = chg

#SVR
chg = []
base = []
final = []
for i in idx:
    t = hemoComposite.loc[i]

    b = 80 * (t['MAP_B']  - t['RAP_B']) / t['CO_B']
    f = 80 * (t['MAP_D']  - t['RAP_D']) / t['CO_D']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
hemoComposite['SVR_B'] = base
hemoComposite['SVR_D'] = final
hemoComposite['SVR_Chg'] = chg

#RAT
chg = []
base = []
final = []
for i in idx:
    t = hemoComposite.loc[i]

    b = t['RAP_B'] / t['PCWP_B']
    f = t['RAP_D'] / t['PCWP_D']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
hemoComposite['RAT_B'] = base
hemoComposite['RAT_D'] = final
hemoComposite['RAT_Chg'] = chg

#PPRatio
chg = []
base = []
final = []
for i in idx:
    t = hemoComposite.loc[i]

    b = t['PP_B'] / t['HRTRT_B']
    f = t['PP_D'] / t['HRTRT_D']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
hemoComposite['PPRatio_B'] = base
hemoComposite['PPRatio_D'] = final
hemoComposite['PPRatio_Chg'] = chg

#PAPi
chg = []
base = []
final = []
for i in idx:
    t = hemoComposite.loc[i]

    b = (t['PAS_B'] - t['PAD_B']) / t['RAP_B']
    f = (t['PAS_D'] - t['PAD_D']) / t['RAP_D']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
hemoComposite['PAPi_B'] = base
hemoComposite['PAPi_D'] = final
hemoComposite['PAPi_Chg'] = chg

#SAPi
chg = []
base = []
final = []
for i in idx:
    t = hemoComposite.loc[i]

    b = (t['BPSYS_B'] - t['BPDIAS_B']) / t['PCWP_B']
    f = (t['BPSYS_D'] - t['BPDIAS_D']) / t['PCWP_D']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
hemoComposite['SAPi_B'] = base
hemoComposite['SAPi_D'] = final
hemoComposite['SAPi_Chg'] = chg

#CPP
chg = []
base = []
final = []
for i in idx:
    t = hemoComposite.loc[i]

    b = t['BPDIAS_B'] - t['PCWP_B']
    f = t['BPDIAS_B'] - t['PCWP_B']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
hemoComposite['CPP_B'] = base
hemoComposite['CPP_D'] = final
hemoComposite['CPP_Chg'] = chg

#PRAPRat
chg = []
base = []
final = []
for i in idx:
    t = hemoComposite.loc[i]

    b = t['PP_B'] / t['RAP_B']
    f = t['PP_D'] / t['RAP_D']
    c = f - b

    base.append(b)
    final.append(f)
    chg.append(c)
    
hemoComposite['PRAPRat_B'] = base
hemoComposite['PRAPRat_D'] = final
hemoComposite['PRAPRat_Chg'] = chg

hemoComposite.head(5)

,RAP_B,PAS_B,PAD_B,PAMN_B,PCWP_B,CO_B,CI_B,SVR_B,MIXED_B,BPSYS_B,...,PAPi_Chg,SAPi_B,SAPi_D,SAPi_Chg,CPP_B,CPP_D,CPP_Chg,PRAPRat_B,PRAPRat_D,PRAPRat_Chg
DEIDNUM,,,,,,,,,,,,,,,,,,,,,
72,24.0,42.0,24.0,30.0,36.0,2.20,1.60,5357.575758,NaN,114.0,...,0.250000,0.777778,1.000000,0.222222,50.0,50.0,0.0,1.166667,1.333333,0.166667
81,10.0,40.0,20.0,27.0,18.0,4.52,2.02,1982.300885,60.0,80.0,...,-0.333333,0.944444,2.352941,1.408497,45.0,45.0,0.0,1.700000,3.333333,1.633333
814,3.0,51.0,19.0,30.0,18.0,4.10,2.00,2621.138211,97.0,96.0,...,2.333333,1.888889,3.500000,1.611111,44.0,44.0,0.0,11.333333,21.000000,9.666667
1084,13.0,40.0,26.0,21.0,20.0,2.91,1.76,3289.805269,NaN,88.0,...,NaN,1.050000,NaN,NaN,47.0,47.0,0.0,1.615385,NaN,NaN
1262,10.0,50.0,25.0,33.0,25.0,2.72,1.43,3274.509804,NaN,90.0,...,0.500000,1.720000,5.125000,3.405000,22.0,22.0,0.0,4.300000,10.250000,5.950000


In [15]:
hemoComposite.to_csv('../Data/Original DataFrames/HemoComposite.csv', index=True)

## Make single point of care values
--> take base and discharge measurements and make them each one row

In [16]:
#Load original dataframes
allDataOrig = pd.read_csv("../Data/Original DataFrames/AllData.csv", sep=",", index_col='DEIDNUM') #all feature dataset
hemoOrig = pd.read_csv("../Data/Original DataFrames/HemoComposite.csv", sep=",", index_col='DEIDNUM') #dataset with only hemodynamics

In [17]:
varz = ['Age', 'Gender', 'Race', 'EjF', 'RAP', 'PAS', 'PAD', 'PAMN', 'PCWP', 'CO', 
                            'CI', 'MIXED', 'BPSYS', 'BPDIAS', 'HRTRT', 'MAP','MPAP', 'CPI',
                            'PP', 'PPP', 'PAPP', 'SVR', 'RAT', 'PPRatio', 'PAPi', 'SAPi', 'CPP', 'PRAPRat']

giantList = []
for i in range(len(hemoOrig)):
    df = hemoOrig.iloc[i]
    row0 = []
    row1 = []
    
    row0.append(df.name)
    row1.append(df.name)
    for v in varz:
    

        row0.append(df[v+"_B"])
        row1.append(df[v+"_D"])

        
    giantList.append(row0)
    giantList.append(row1)


varz.insert(0, 'DEIDNUM')
hemoSingleDF = pd.DataFrame(giantList, columns=varz).set_index('DEIDNUM', drop=True)

In [18]:
hemoSingleDF.head(8)

,Age,Gender,Race,EjF,RAP,PAS,PAD,PAMN,PCWP,CO,...,PP,PPP,PAPP,SVR,RAT,PPRatio,PAPi,SAPi,CPP,PRAPRat
DEIDNUM,,,,,,,,,,,,,,,,,,,,,
72,88.0,2.0,1.0,25.0,24.0,42.0,24.0,30.0,36.0,2.20,...,28.0,0.245614,0.428571,5357.575758,0.666667,0.266667,0.750000,0.777778,50.0,1.166667
72,88.0,2.0,1.0,25.0,18.0,42.0,24.0,30.0,24.0,3.10,...,24.0,0.235294,0.428571,3509.677419,0.750000,0.272727,1.000000,1.000000,50.0,1.333333
81,69.0,1.0,1.0,20.0,10.0,40.0,20.0,27.0,18.0,4.52,...,17.0,0.212500,0.500000,1982.300885,0.555556,0.242857,2.000000,0.944444,45.0,1.700000
81,69.0,1.0,1.0,20.0,12.0,35.0,15.0,25.0,17.0,4.65,...,40.0,0.470588,0.571429,1772.043011,0.705882,0.571429,1.666667,2.352941,45.0,3.333333
814,58.0,1.0,1.0,NaN,3.0,51.0,19.0,30.0,18.0,4.10,...,34.0,0.354167,0.627451,2621.138211,0.166667,0.566667,10.666667,1.888889,44.0,11.333333
814,58.0,1.0,1.0,NaN,2.0,38.0,12.0,21.0,12.0,NaN,...,42.0,0.446809,0.684211,NaN,0.166667,0.700000,13.000000,3.500000,44.0,21.000000
1084,49.0,2.0,2.0,14.0,13.0,40.0,26.0,21.0,20.0,2.91,...,21.0,0.238636,0.350000,3289.805269,0.650000,0.190909,1.076923,1.050000,47.0,1.615385
1084,49.0,2.0,2.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.0,NaN


In [19]:
#save to file
hemoSingleDF.to_csv("../Data/Original DataFrames/HemoSingleValue.csv")

## All Data

In [20]:
giantList = []
for i in range(len(allDataOrig)):
    df = allDataOrig.iloc[i]
    row0 = []
    row1 = []
    row0.append(df.name)
    row1.append(df.name)
    for col in df.index:
        if "_B" in col:
            row0.append(df[col])
        elif "_D" in col:
            row1.append(df[col])
        elif "_Chg" in col or "_M3" in col or "_M6" in col:
            pass
        else: #value for both base and discharge
            row0.append(df[col])
            row1.append(df[col])
    
    giantList.append(row0)
    giantList.append(row1)

    

allDataSingleDF = pd.DataFrame(giantList, columns=['DEIDNUM','Age', 'Gender', 'Race', 'Wt', 'BMI', 
       'InitialHospDays', 'TotalHospDays', 'NYHA', 'MLHFS', 'AF',
       'AlchE', 'ANGP', 'ARRH', 'CARREST', 'CVD', 'COPD', 'DEPR', 'DIAB',
       'GOUT', 'HEPT', 'HTN', 'MALIG', 'RENAL', 'SMOKING', 'STERD',
       'StrokeTIA', 'VAHD', 'VF', 'VHD', 'VT', 'ISCH', 'NonISCH', 'CABG',
       'HTRANS', 'ICD', 'PACE', 'PTCI', 'SixFtWlk', 'VO2','ALB', 'ALT', 'AST', 'BUN',
       'CRT', 'DIAL', 'HEC', 
       'HEM','PLA', 'POT',  'SOD',
       'TALB', 'TOTP', 'WBC', 'ACE','BET', 'NIT', 'DIUR',
       'EjF', 'BPDIAS', 'BPSYS', 'HR',
       'PV', 'MAP', 'PP', 'PPP', 'PPRatio']
).set_index('DEIDNUM', drop=True)

In [21]:
allDataSingleDF.head(8)

,Age,Gender,Race,Wt,BMI,InitialHospDays,TotalHospDays,NYHA,MLHFS,AF,...,DIUR,EjF,BPDIAS,BPSYS,HR,PV,MAP,PP,PPP,PPRatio
DEIDNUM,,,,,,,,,,,,,,,,,,,,,
72,88.0,2.0,1.0,57.100000,26.424175,9.0,16.0,4.0,76.0,1.0,...,1.0,25.0,68.0,94.0,104.0,12.452563,139.333333,26.0,0.276596,0.250000
72,88.0,2.0,1.0,52.000000,24.064047,9.0,16.0,3.0,76.0,1.0,...,1.0,25.0,60.0,86.0,92.0,12.452563,126.000000,26.0,0.302326,0.282609
81,69.0,1.0,1.0,102.363636,32.307675,5.0,13.0,4.0,21.0,1.0,...,1.0,20.0,56.0,88.0,70.0,-3.346568,125.333333,32.0,0.363636,0.457143
81,69.0,1.0,1.0,100.454545,31.705134,5.0,13.0,2.0,21.0,1.0,...,1.0,20.0,43.0,77.0,72.0,-3.346568,105.666667,34.0,0.441558,0.472222
86,56.0,2.0,2.0,98.636364,40.016375,3.0,3.0,4.0,NaN,0.0,...,1.0,25.0,70.0,100.0,86.0,5.478621,146.666667,30.0,0.300000,0.348837
86,56.0,2.0,2.0,97.000000,39.352509,3.0,3.0,4.0,NaN,0.0,...,1.0,25.0,56.0,97.0,82.0,5.478621,134.333333,41.0,0.422680,0.500000
267,82.0,1.0,1.0,82.200000,30.938312,15.0,14.0,4.0,60.0,1.0,...,1.0,25.0,54.0,96.0,70.0,-29.218833,132.000000,42.0,0.437500,0.600000
267,82.0,1.0,1.0,81.700000,30.750122,15.0,14.0,4.0,60.0,1.0,...,1.0,25.0,60.0,100.0,100.0,-29.218833,140.000000,40.0,0.400000,0.400000


In [22]:
#save to file
allDataSingleDF.to_csv("../Data/Original DataFrames/AllDataSingleValue.csv")